# Explore here

In [70]:
# Your code here
import pandas as pd
import sqlite3 
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors


In [71]:
pd.set_option('display.max_rows', 10)

movies = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv")
movies

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [72]:
credits=pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv")
credits

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [73]:
con=sqlite3.connect("moviesDB")

movies.to_sql("movies",con,if_exists="replace",index=False)
credits.to_sql("credits",con,if_exists="replace",index=False)


4803

In [74]:
query="SELECT * FROM movies INNER JOIN credits ON movies.title==credits.title"

data=pd.read_sql_query(query,con)
data

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",None,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992

In [75]:
con.close()

In [76]:


data.drop_duplicates()
data = data.loc[:, ~data.columns.duplicated()]
data=data[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
data


,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [77]:
def extractJson(x):
    try:
        return json.loads(x)
    except (ValueError, TypeError):
        return None
    

In [78]:
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_rows', None)

data.loc[:, 'genres'] =data['genres'].apply(lambda x: [item["name"] for item in  extractJson(x)] if pd.notna(x) else None)
data.loc[:, 'genres'] =data['genres'].apply(lambda x: [str(genre) for genre in x])
data= data[data['genres'].apply(lambda x: x != [])]
data['genres']


0           [Action, Adventure, Fantasy, Science Fiction]
1                            [Adventure, Fantasy, Action]
2                              [Action, Adventure, Crime]
3                        [Action, Crime, Drama, Thriller]
4                    [Action, Adventure, Science Fiction]
5                            [Fantasy, Action, Adventure]
6                                     [Animation, Family]
7                    [Action, Adventure, Science Fiction]
8                            [Adventure, Fantasy, Family]
9                            [Action, Adventure, Fantasy]
10          [Adventure, Fantasy, Action, Science Fiction]
11                   [Adventure, Action, Thriller, Crime]
12                           [Adventure, Fantasy, Action]
13                           [Action, Adventure, Western]
14          [Action, Adventure, Fantasy, Science Fiction]
15                           [Adventure, Family, Fantasy]
16                   [Science Fiction, Action, Adventure]
17            

In [79]:
data.loc[:, 'keywords']=data['keywords'].apply(lambda x: [item["name"] for item in  extractJson(x)] if pd.notna(x) else None)
data.loc[:, 'keywords']=data['keywords'].apply(lambda x: [str(keyword) for keyword in x])
data=data[data['keywords'].apply(lambda x: x != [])]
data['keywords']


0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
5       [dual identity, amnesia, sandstorm, love of on...
6       [hostage, magic, horse, fairy tale, musical, p...
7       [marvel comic, sequel, superhero, based on com...
8       [witch, magic, broom, school of witchcraft, wi...
9       [dc comics, vigilante, superhero, based on com...
10      [saving the world, dc comics, invulnerability,...
11      [killing, undercover, secret agent, british se...
12      [witch, fortune teller, bondage, exotic island...
13      [texas, horse, survivor, texas ranger, partner...
14      [saving the world, dc comics, superhero, based...
15      [based on novel, fictional place, brother sist...
16      [new york, shield, marvel comic, superhero, ba...
17      [sea, 

In [80]:
data.loc[:, 'cast']=data['cast'].apply(lambda x: [item["name"] for item in  extractJson(x)][:3] if pd.notna(x) else None)
data.loc[:, 'cast']=data['cast'].apply(lambda x: [str(cast) for cast in x])
data=data[data['cast'].apply(lambda x: x != [])]
data['cast']

0        [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1           [Johnny Depp, Orlando Bloom, Keira Knightley]
2            [Daniel Craig, Christoph Waltz, Léa Seydoux]
3            [Christian Bale, Michael Caine, Gary Oldman]
4          [Taylor Kitsch, Lynn Collins, Samantha Morton]
5            [Tobey Maguire, Kirsten Dunst, James Franco]
6               [Zachary Levi, Mandy Moore, Donna Murphy]
7       [Robert Downey Jr., Chris Hemsworth, Mark Ruff...
8           [Daniel Radcliffe, Rupert Grint, Emma Watson]
9                  [Ben Affleck, Henry Cavill, Gal Gadot]
10           [Brandon Routh, Kevin Spacey, Kate Bosworth]
11        [Daniel Craig, Olga Kurylenko, Mathieu Amalric]
12          [Johnny Depp, Orlando Bloom, Keira Knightley]
13          [Johnny Depp, Armie Hammer, William Fichtner]
14             [Henry Cavill, Amy Adams, Michael Shannon]
15         [Ben Barnes, William Moseley, Anna Popplewell]
16         [Robert Downey Jr., Chris Evans, Mark Ruffalo]
17            

In [81]:
data.loc[:, 'overview']=data["overview"].apply(lambda x: [x])
data.loc[:, 'overview']=data['overview'].apply(lambda x: [str(overview) for overview in x])
data=data[data['overview'].apply(lambda x: x != [])]
data['overview']

0       [In the 22nd century, a paraplegic Marine is d...
1       [Captain Barbossa, long believed to be dead, h...
2       [A cryptic message from Bond’s past sends him ...
3       [Following the death of District Attorney Harv...
4       [John Carter is a war-weary, former military c...
5       [The seemingly invincible Spider-Man goes up a...
6       [When the kingdom's most wanted-and most charm...
7       [When Tony Stark tries to jumpstart a dormant ...
8       [As Harry begins his sixth year at Hogwarts, h...
9       [Fearing the actions of a god-like Super Hero ...
10      [Superman returns to discover his 5-year absen...
11      [Quantum of Solace continues the adventures of...
12      [Captain Jack Sparrow works his way out of a b...
13      [The Texas Rangers chase down a gang of outlaw...
14      [A young boy learns that he has extraordinary ...
15      [One year after their incredible adventures in...
16      [When an unexpected enemy emerges and threaten...
17      [Capta

In [82]:
data.loc[:, 'crew']=data["crew"].apply(lambda x: " ".join([item['name'] for item in extractJson(x) if item['job'] == 'Director']))
data.loc[:, 'crew']=data['crew'].apply(lambda x: [str(name) for name in x])
data=data[data['crew'].apply(lambda x: x != [])]
data['crew']

0                 [J, a, m, e, s,  , C, a, m, e, r, o, n]
1              [G, o, r, e,  , V, e, r, b, i, n, s, k, i]
2                          [S, a, m,  , M, e, n, d, e, s]
3       [C, h, r, i, s, t, o, p, h, e, r,  , N, o, l, ...
4              [A, n, d, r, e, w,  , S, t, a, n, t, o, n]
5                             [S, a, m,  , R, a, i, m, i]
6       [B, y, r, o, n,  , H, o, w, a, r, d,  , N, a, ...
7                       [J, o, s, s,  , W, h, e, d, o, n]
8                       [D, a, v, i, d,  , Y, a, t, e, s]
9                       [Z, a, c, k,  , S, n, y, d, e, r]
10                   [B, r, y, a, n,  , S, i, n, g, e, r]
11                   [M, a, r, c,  , F, o, r, s, t, e, r]
12             [G, o, r, e,  , V, e, r, b, i, n, s, k, i]
13             [G, o, r, e,  , V, e, r, b, i, n, s, k, i]
14                      [Z, a, c, k,  , S, n, y, d, e, r]
15             [A, n, d, r, e, w,  , A, d, a, m, s, o, n]
16                      [J, o, s, s,  , W, h, e, d, o, n]
17            

In [83]:
print(data["overview"].head())
print(data["genres"].head())
print(data["keywords"].head())
print(data["cast"].head())
print(data["crew"].head())

0    [In the 22nd century, a paraplegic Marine is d...
1    [Captain Barbossa, long believed to be dead, h...
2    [A cryptic message from Bond’s past sends him ...
3    [Following the death of District Attorney Harv...
4    [John Carter is a war-weary, former military c...
Name: overview, dtype: object
0    [Action, Adventure, Fantasy, Science Fiction]
1                     [Adventure, Fantasy, Action]
2                       [Action, Adventure, Crime]
3                 [Action, Crime, Drama, Thriller]
4             [Action, Adventure, Science Fiction]
Name: genres, dtype: object
0    [culture clash, future, space war, space colon...
1    [ocean, drug abuse, exotic island, east india ...
2    [spy, based on novel, secret agent, sequel, mi...
3    [dc comics, crime fighter, terrorist, secret i...
4    [based on novel, mars, medallion, space travel...
Name: keywords, dtype: object
0    [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1       [Johnny Depp, Orlando Bloom, Keira Knightley]

In [84]:
data["tags"] = data["overview"] + data["genres"] + data["keywords"] + data["cast"] + data["crew"]
data["tags"] = data["tags"].apply(lambda x: ",".join(x).replace(",", " "))

data.drop(columns = ["genres", "keywords", "cast", "crew", "overview"], inplace = True)



In [85]:
vector=TfidfVectorizer()
data_Vectorized=vector.fit_transform(data['tags'])



In [86]:
model=NearestNeighbors(n_neighbors=6,algorithm="brute",metric="cosine")
model.fit(data_Vectorized)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [108]:
def recommend(movie):
    movie_index = data[data["title"] == movie].index[0]
    distances, indexs = model.kneighbors(data_Vectorized[movie_index])
    recommendations = []
    for i in range(0, len(distances[0])):  # Evitar la primera porque es la misma película
        movie_title = data.iloc[indexs[0][i]].title
        recommendations.append((movie_title, distances[0][i])) 
    return recommendations


In [114]:
input_movie = 'Captain America: Civil War'
recommendations=recommend(input_movie)
for movie, distance in recommendations:
    print(f"- Pelicula: {movie}, Distancia: {distance} ")

- Pelicula: Avengers: Age of Ultron, Distancia: 0.6512374553521478 
- Pelicula: Captain America: The Winter Soldier, Distancia: 0.6745914456099176 
- Pelicula: The Avengers, Distancia: 0.6811493459231479 
- Pelicula: Iron Man 3, Distancia: 0.6877486379300684 
- Pelicula: Captain America: The First Avenger, Distancia: 0.7039241336378157 
